In [3]:
# %%writefile imports.py
import pandas as pd
import numpy as np
from os.path import join
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from itertools import cycle, chain
import seaborn as sns
import glob
from scipy import stats

sns.set(font_scale=1.2)
pd.set_option('max_rows', 100)
MARKER = 'ods.^*'

COLORS = sns.color_palette('Set1', 12, desat=0.5)
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color=COLORS)
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 15
mpl.rcParams['axes.labelsize'] = 15

%matplotlib inline
%config InlineBackend.figure_format='retina'
from pandas.plotting import scatter_matrix
from pydoc import help
from scipy.stats.stats import pearsonr
# import plotStdScan
# import imp
# imp.reload(plotStdScan)
import pyodbc
from sqlalchemy import create_engine
import importlib

import tensorflow as tf

%load_ext memory_profiler
%memit


peak memory: 245.87 MiB, increment: 0.04 MiB


In [2]:
sc.stop()

In [3]:
from pyspark import SparkContext
sc =SparkContext()
# sc =SparkContext.getOrCreate()

In [ ]:
from urllib.request import urlretrieve
urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

In [7]:
import pyspark

In [4]:
data_file = "kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

In [5]:
raw_data.count()

494021

In [29]:
sc2.stop()

In [31]:
from pyspark.context import SparkContext,SparkConf
# conf1 = SparkConf().setAppName('testing1').setMaster('local')
# conf2 = SparkConf().setAppName('testing2').setMaster('local')
sc = SparkContext('local', 'test1')
sc.stop() #Cannot run multiple SparkContexts at once; 
# must stop the open one first to start the second one.
sc = SparkContext('local', 'test2')
# sc.stop()

In [32]:
sc.appName

'test2'

## 1. The filter transformation
This transformation can be applied to RDDs in order to keep just elements that satisfy a certain condition. More concretely, a function is evaluated on every element in the original RDD. The new resulting RDD will contain just those elements that make the function return True.
For example, imagine we want to count how many normal. interactions we have in our dataset. We can filter our raw_data RDD as follows.

**1.1 From blow timing we can see it only takes 75us which is the overhead time, not really filtering process**

In [12]:
# %%timeit
normal_raw_data = raw_data.filter(lambda x: 'normal' in x)

## 2. Action: count()

##### count partitions 

In [13]:
normal_raw_data.getNumPartitions()

1

## 2.1 %%timeit does not memorize the name normal_count
* Do it without %%timeit if you want it to be in memory

In [14]:
%%timeit #will run 8 times
normal_count = normal_raw_data.count()
print(normal_count)

97278
97278
97278
97278
97278
97278
97278
97278
526 ms ± 8.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
normal_count = normal_raw_data.count()
print(normal_count)

97278


## 3. raw_data count takes less time since normal_raw_data need to do one more step:  filtering

In [16]:
type(raw_data), type(normal_raw_data)

(pyspark.rdd.RDD, pyspark.rdd.PipelinedRDD)

In [17]:
# %%timeit
raw_count = raw_data.count()
print(raw_count)

494021


## 4. Turn RDD to PipelineRDD by transformation: map, filter, etc..

In [19]:
mp_raw_data = raw_data.map(lambda x: x)
type(mp_raw_data)

pyspark.rdd.PipelinedRDD

In [20]:
type(mp_raw_data)

pyspark.rdd.PipelinedRDD

### time the operations: filter
* time 1000 for ms
* default is by second

In [21]:
from time import time
t0 = time()
normal_count = normal_raw_data.count()
tt = time() - t0
tt*1000

546.8239784240723

In [22]:
%%timeit
normal_count = normal_raw_data.count()

528 ms ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 5. Turn PipelineRDD into RDD

In [23]:
back_raw_data = mp_raw_data.collect()

In [24]:
type(back_raw_data)

list

In [25]:
back_raw_data[:5]

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.']

In [26]:
df = pd.DataFrame(back_raw_data[:5])
df

,0
0,"0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0..."
1,"0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,..."
2,"0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0..."
3,"0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0..."
4,"0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0..."


## Try to partition collected data


* Use numSlices to partition: sc.parallelize(back_raw_data,numSlices=8)
* If using sc.parallelize(back_raw_data).partitionBy(9), then can not count()

In [27]:
rdd = sc.parallelize(back_raw_data).partitionBy(9)
rdd.getNumPartitions()

9

#### Give error to count

rdd.count() # Give error

#### Use numSlices to partition is okay, w/o errors

In [29]:
rdd = sc.parallelize(back_raw_data,numSlices=8)

In [30]:
rdd.getNumPartitions()

8

In [31]:
rdd.count()

494021

In [32]:
type(rdd),type(raw_data)

(pyspark.rdd.RDD, pyspark.rdd.RDD)

In [33]:
%%timeit
rdd_data = rdd.collect()

727 ms ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
# %%timeit
rdd.count()

494021

## github reference:
https://github.com/jadianes/spark-py-notebooks